Goals

be able to access all data in S3 for a given season
- events table
- player and team match stats
- lineups and missing players table
- odds table


our end goal is to have player and team tables for the season which will facilitate making our features very easily. 
that means we should have a player table of every performance in the league with vaep, xG, rest days (have to incorporate european fixtures), travel distance (have to manually get coordinates for stadiums),  

then wrangle event data to get 

In [ ]:
import boto3
from dotenv import load_dotenv
import os
import warnings
from io import StringIO
import pandas as pd
import socceraction.spadl as spadl

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
warnings.filterwarnings('ignore')

load_dotenv()
aws_access_key = os.getenv('AWS_ACCESS_KEY')
aws_secret_access = os.getenv('AWS_SECRET_ACCESS')
aws_region = os.getenv('AWS_REGION')

s3 = boto3.client('s3',
                aws_access_key_id=aws_access_key,
                aws_secret_access_key=aws_secret_access,
                region_name=aws_region)

bucket = 'footballbets'
league = "FRA-Ligue 1"
season = 2223

In [ ]:
s3_event = s3.get_object(Bucket=bucket, Key=f'{league}/{season}/events_spadl.csv')
event_data = pd.read_csv(StringIO(s3_event['Body'].read().decode('utf-8')))

In [ ]:
event_data.head()

In [ ]:
event_data.type.unique()

In [ ]:
[item for item in event_data[event_data['type'] == 'Pass'].qualifiers]

In [ ]:
next_event = event_data.shift(-1, fill_value=0)


In [ ]:
spadl_e = s3.get_object(Bucket=bucket, Key=f'ENG-Premier League/2223/events_spadl.csv')
spadldf = pd.read_csv(StringIO(spadl_e['Body'].read().decode('utf-8')))
spadldf = spadl.add_names(spadldf)

In [ ]:
sample[(sample['nextEvent'].isin(['interception', 'tackle'])) & (sample['result_name'] == 'success')].type_name.value_counts()

In [ ]:
sample.result_name.value_counts()

In [ ]:
sample = spadldf[spadldf['game_id'] == 1640952]

In [ ]:
next_event = sample.shift(-1, fill_value=0)
sample["nextEvent"] = next_event["type_name"]
sample['kickedOut'] = sample.apply(lambda x: 1 if x['nextEvent'] == 'throw_in' else 0, axis=1)

sample['nextTeamId'] = next_event['team_id']
chain_team = sample.iloc[0]['team_id']
period = sample.iloc[0]['period_id']

stop_criterion = 0
chain = 0
sample['possession_chain'] = 0
sample['possession_chain_team'] = 0

for i, row in sample.iterrows():
    sample.at[i, 'possession_chain'] = chain
    sample.at[i, 'possession_chain_team'] = chain_team

    if row.type_name in ['pass', 'duel', 'dribble']:

        if row['result_name'] == 'fail':
            if row.nextEvent == 'interception' or row.nextEvent == 'tackle':
                stop_criterion += 2
            else:
                stop_criterion +=1

        if row.team_id != row.nextTeamId:
            if row.nextEvent == 'dribble' or row.nextEvent == 'pass' or row.nextEvent == 'tackle':
                stop_criterion += 2

    # if row.type_name == 'interception' or row.type_name == 'tackle':
    #     if row.result_name == 'success':
    #         stop_criterion += 2

    if row.type_name in ['pass', 'cross', 'freekick_crossed', 'corner_crossed']:
        if row.result_name == 'offside':
            stop_criterion += 2
    if row.type_name in ['shot', 'foul', 'clearance']:
        stop_criterion += 2
    if row['kickedOut'] == 1:
        stop_criterion += 2

    if row['period_id'] != period:
        chain += 1
        stop_criterion = 0
        chain_team = row['team_id']
        period = row['period_id']
        sample.at[i, 'possession_chain'] = chain
        sample.at[i, 'possession_chain_team'] = chain_team
    
    if stop_criterion >= 2:
        chain += 1
        stop_criterion = 0
        chain_team = row['nextTeamId']


In [ ]:
sample[sample['possession_chain'] == 19]

In [ ]:
import soccerdata as sd
fbref = sd.FBref(leagues=league, seasons=season)

In [ ]:
schedule = fbref.read_schedule()

In [ ]:
schedule.iloc[:380].index[0]